# Star Resonance Tool - Colab Launcher

このノートブックは、Google Colab上で「Star Resonance Tool」を実行するためのものです。
以下のセルを順番に実行してください。

### 手順
1. 再生ボタン(▶)を押してセルを実行します。
2. IPアドレスが表示されるので、コピーしておきます（例: `34.123.45.67`）。
3. `your url is: ...` というリンクが表示されたらクリックします。
4. コピーしたIPアドレスを入力して `Click to Submit` を押します。
5. アプリが起動します。

In [ ]:
# リポジトリのクローン
!git clone https://github.com/takeshich/star-resonance-tool.git
%cd star-resonance-tool

# 依存関係のインストール
!pip install -r requirements.txt
!pip install pyngrok

# localtunnelのインストール
!npm install -g localtunnel

In [ ]:
import urllib

# IPアドレスの取得と表示
print("Password/Endpoint IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

# Streamlitをバックグラウンドで実行
get_ipython().system_raw('streamlit run app.py --server.enableCORS=false --server.enableXsrfProtection=false &')

# localtunnelでポート開放
!npx localtunnel --port 8501